<a href="https://colab.research.google.com/github/Kgisb/laiqa/blob/main/Manual_Pilot_Nov28_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages')
!pip install fuzzywuzzy
from fuzzywuzzy import process
from collections import Counter
import re
import numpy as np


Complete_file = 'Complete_results_file_nov28.xlsx'

ayurveda_mapping = {
    "define your menstrual cycle": {
        "Irregular": "Vata",
        "Regular": ["Pitta", "Kapha"],
        "Atypical": "VPK"
    },
    "frequency of your menstruation": {
        "It comes less than every 20 days": "Vata",
        "Usually it comes after 35 days": "Pitta",
        "It comes in regular intervals": "Kapha",
        "Not sure/ not observerd": "VPK"
    },
    "bleed between periods": {
        "Sometimes": "Vata",
        "Never": ["Kapha", "Pitta"]
    },
    "your period cycle": {
        "26 -28 (Normal)": "Vata",
        "28 - 30 (Normal)": "Pitta",
        "30 - 35 (Normal)": "Kapha",
        "Atypical": "VPK"
    },
    "colour of your period blood": {
        "Menstrual blood is frothy, thin, rough, blackish or reddish": "Vata",
        "Menstrual blood is blue, yellow or blackish in color": "Pitta",
        "Blood resembles the water mixed with red ochre": "Kapha",
        "Blueish and Bronze color": "VPK",
        "Resembles like Kanji": "VPK"
    },
    "texture of your period blood": {
        "Small quantity & irregular": "Vata",
        "Heavy and repeated": "Pitta",
        "Excreted very slowly": "Kapha",
        "Atypical secretion": "VPK"
    },
    "odour of your period": {
        "Iron like odor": "Vata",
        "Having musty or fishy smell": "Pitta",
        "Menstrual blood is slimy, pale, heavy, unctuous, cold, mixed with mucus and thick, discharge": "Kapha",
        "Severely Foul Smell": "VPK"
    },
    "spotted clots in your flow": {
        "Does not clot": ["Vata", "Pitta"],
        "Get clotted like a muscle fibers": "Kapha",
        "Atypical clots": "VPK"
    },
    "burn during your flow": {
        "Mild": "Vata",
        "Moderate": "Pitta",
        "No": "General",
        "Severe": "VPK"
    },
    "frequency of your pain": {
        "Sometimes pain sometime no pain": "Vata",
        "Substantial pain with heat": "Pitta",
        "Mild pain": "Kapha",
        "Moderate": "VPK"
    },
    "kind of cramps you feel": {
        "Mild cramps": "Vata",
        "Moderate cramps": "Pitta",
        "Mild Cramps with Heaviness": "Kapha",
        "Heavy Cramps": "VPK"
    },
    "get fever during periods": {
        "No": ["Vata", "Pitta", "Kapha"],
        "Yes": "VPK"
    },
    "are you anemic": {
        "No": ["Vata", "Pitta", "Kapha"],
        "Yes": "VPK"
    },
    "excessive and persistent thirst": {
        "No": ["Vata", "Pitta", "Kapha"],
        "Yes": "VPK"
    },
    "suffer from headache during periods": {
        "Yes": ["Vata", "Pitta", "Kapha"],
        "No": ["Vata", "Pitta", "Kapha"]
    },
    "your anxious levels": {
        "Moderate": "Vata",
        "Mild": "Pitta",
        "No": ["Vata", "Pitta", "Kapha"],
        "Severe": "VPK"
    },
    "feel irritated during your periods": {
        "Mild": "Vata",
        "Moderate": "Pitta",
        "No": ["Vata", "Pitta", "Kapha"],
        "Severe": "VPK"
    },
    "feel stressed": {
        "Mild": "Vata",
        "Moderate": ["Pitta", "VPK"],
        "No": ["Vata", "Pitta", "Kapha"]
    },
    "find it difficult to sleep": {
        "Moderate": "Vata",
        "Mild": "Pitta",
        "No": ["Vata", "Pitta", "Kapha"],
        "Severe": "VPK"
    },
    "get fungal or yeast infection frequently": {
        "No": ["Vata", "Pitta", "Kapha"],
        "Mild": "Kapha",
        "Moderate": "VPK"
    }
}


# Prakriti mapping dictionary
prakriti_mapping = {
    'slim': 'Vata',
    'athletic': 'Pitta',
    'broad': 'Kapha',
    'petite': 'Vata',
    'moderate': 'Pitta',
    'strong': 'Kapha',
    'dry and cracked': 'Vata',
    'soft, thin, oily, acne prone': 'Pitta',
    'smooth, firm and clear': 'Kapha',
    'thin, dry, brittle': 'Vata',
    'thin, oily, early graying': 'Pitta',
    'thick, smooth, and firm': 'Kapha',
    'by nature i have a thin frame': 'Vata',
    'i have the tendency to fluctuate': 'Pitta',
    'tend to get obese': 'Kapha',
    'small portion and frequent': 'Vata',
    'normal portion and fixed time': 'Pitta',
    'high quantity and fixed time': 'Kapha',
    'irregular bowel movements': 'Vata',
    'regular, two or more times': 'Pitta',
    'fixed and good quantity': 'Kapha',
    'brisk': 'Vata',
    'less mobile': 'Kapha',
    'intolerance to cold weather': 'Vata',
    'intolerance to hot weather': 'Pitta',
    'tolerant to both heat and cold': 'Kapha',
    'poor': 'Vata',
    'good': 'Vata',
    'excellent': 'Kapha',
    'frequent': 'Vata',
    'low digestive': 'Pitta',
    'irregular': 'Kapha',
    'low': 'Kapha',
    'tend to talk a lot': 'Vata',
    'strong in communication and analytical abilities': 'Pitta',
    'strong in communication but talk less': 'Kapha',
    'enthusiastic': 'Vata',
    'with less conviction': 'Pitta',
    'not very enthusiastic': 'Kapha',
    'i feel like aging fast': 'Vata',
    'halfway there': 'Pitta',
    'i look younger than my age': 'Kapha',
    'developmental, neurological, dementia, movement and speech disorders, arrhythmias': 'Vata',
    'ulcer, bleeding disorders, skin diseases': 'Pitta',
    'obesity, diabetes, atherosclerotic conditions': 'Kapha'
}

# MPA Score Mapping Dictionary

mpa_score_mapping = {
    'none': 0.0,
    'Not at all': 0.0,
    'almost never': 1.0,
    'Rarely': 1.0,
    'almost always': 2.0,
    'always': 3.0,
    'no part of body': 0.0,
    'lower abdomen': 1.0,
    'lumbar region': 2.0,
    'lower limbs': 2.0,
    'inguinal region': 3.0,
    'does not hurt': 0.0,
    'hurts a little': 1.0,
    'hurts a little more': 2.0,
    'hurts even more': 2.0,
    'hurts a lot': 3.0,
    'hurts a lot more': 3.0,
    'mild': 0.0,
    'moderate': 1.0,
    'severe': 2.0,
    '0.0': 0.0,
    '1-2 days': 0.0,
    '3-4 days': 1.0,
    '5 days or more': 2.0,
    '2023-01-02 00:00:00': 1.0,
    '2023-03-04 00:00:00': 2.0,
    'Usually (on most occasions)':2.0,
    '>=5': 3.0
}

# MWQ Score Mapping Dictionary
mwq_mappings = {
    'applies to me to a severe degree': 0.0,
    'applies to me to a considerable degree': 1.0,
    'applies to me to a moderate degree': 2.0,
    'applies to me to a mild degree': 3.0,
    'does not apply to me': 4.0,
    "It doesn’t bother me much": 4.0,
    "It's mild": 3.0,
    "It's moderate": 2.0,
    "It's quite considerable": 1.0,
    "It's severe": 0.0,
    "Intensely": 0.0,
    "Moderately": 1.0,
    "Mostly": 2.0,
    "Infrequently": 3.0,
    "Not at all": 4.0,
    "5 - Very often": 0.0,
    "4 - Often": 1.0,
    "3 - Sometimes": 2.0,
    "2 -  Rarely": 3.0,
    "1 - Never": 4.0,
    "Significantly": 0.0,
    "Quite considerably": 1.0,
    "Moderately that's noticeable": 2.0,
    "Slightly": 3.0,
    "Not really": 4.0,
    "I have severe skin related issues and symptoms": 0.0,
    "It is quite bothersome": 1.0,
    "Moderately bothersome": 2.0,
    "Slightly bothersome and noticeable": 3.0

}

# Define the text-to-code mapping
text_to_code_mapping = {
    'Never': 0,
    'Almost Never': 1,
    'Sometimes': 2,
    'Often': 3,
    'Usually': 4
}

score_mapping_occupation = {
    1: 0,   # 1 maps to 0
    2: 1,   # 2 maps to 1
    3: 1.5, # 3 maps to 1.5
    4: 3.5, # 4 maps to 3.5
    5: 6.0  # 5 maps to 6.0
}

score_mapping_pss = {
    0: 0,  # 0 maps to 0
    1: 1,  # 1 maps to 1
    2: 2,  # 2 maps to 2
    3: 3,  # 3 maps to 3
    4: 4   # 4 maps to 4
}


"""
# Reverse the mapping to create a new dictionary
reverse_mapping = {v: 4 - v for v in text_to_code_mapping.values()}

# Define the columns that require reverse mapping
columns_to_reverse = [
    'How often you felt confident handling tricky situations?',
    'How often you felt things happened according to your plans?',
    'Were you able to manage your frustrations?',
    'Did you feel you are on top of things and had everything under control?'
]
"""
# Define the column ranges for each category
CATEGORY_RANGES = {
    'Pain Score': 'CE:CJ',
    'Concentration': 'CK:CR',
    'Behavioral Changes': 'CS:CW',
    'Autonomic Reactions': 'CX:DA',
    'Water Retention': 'DB:DF',
    'Negative Emotions': 'DG:DM',
    'Arousal': 'DN:DR',
    'Control': 'DS:DX',
    'Appetite': 'DY:DY'
}

sleep_map = {
    0:0,
    1:5,
    2:15
}


#Free plan functions

def calculate_sleep_score(row):
    score_mapping = {
        'Always': 2,
        "Never - I fall back asleep immediately": 2,
        "No, i am mostly sound asleep": 2,
        "I usually get a good sound sleep": 2,
        'Sometimes': 1,
        "Sometimes it takes me more than 30 minutes but other times, i sleep quickly": 1,
        "Moderately (Sometimes i don't feel like i had a good sleep)": 1,
        'Never': 0,
        "Yes, very frequently": 0,
        "Not much (I usually don't get a good night relaxing sleep)": 0,
        "Almost always, once my sleep is interrupted, it takes me more than 30 min to go back to sleep": 0
    }

    sleep_map = {
        0: 0,
        1: 5,
        2: 15
    }

    # Calculate the total sleep score
    total_sleep = sum(score_mapping.get(item, 0) for item in row)

    # Map the total sleep score using sleep_map
    mapped_sleep_score = sleep_map.get(total_sleep, 0)  # Default to 0 if total_sleep is not in sleep_map

    max_sleep = 75
    global sleep_pt; sleep_pt = (mapped_sleep_score/max_sleep) * 100

    return mapped_sleep_score, sleep_pt

def calculate_bmi(height_ft, height_in, weight_kg):
    height_m = height_ft * 0.3048 + height_in * 0.0254
    print(f'the height is {height_m} in meters')
    print(f'The weight is {weight_kg}')
    if height_m != 0:
        BMI = weight_kg / (height_m ** 2)
        print(f'BMI is {BMI}')
        return BMI
    else:
        print('Some bug in BMI calculation')


def calculate_occupational_stress(row, text_to_code_mapping, score_mapping):
    # Convert text responses to numeric codes, handle unmapped responses with a default value (e.g., 0)
    converted_row = row.map(lambda x: text_to_code_mapping.get(x, 0))

    # Map the converted responses to their corresponding scores
    scored_row = converted_row.map(lambda x: score_mapping_occupation.get(x, 0))

    # Calculate the total score
    total_score = converted_row.sum()

    total_score_forpt = scored_row.sum()

    # Calculate the percentage
    max_possible_score = 48  # 8 questions * highest score per question (6)
    global occ_pt; occ_pt = (total_score_forpt / max_possible_score) * 100

    # Calculate the category based on the total score
    if total_score <= 15:
        category = "Relatively calm"
    elif 16 <= total_score <= 20:
        category = "Fairly low"
    elif 21 <= total_score <= 25:
        category = "Moderate"
    elif 26 <= total_score <= 30:
        category = "Severe stress"
    elif 31 <= total_score <= 40:
        category = "Stress level is potentially dangerous"
    else:
        category = "Unknown"

    return category, occ_pt



def calculate_perceived_stress(row, text_to_code_mapping, score_mapping_pss):
    # Convert the row using the text-to-code mapping
    converted_row = row.map(text_to_code_mapping)

    scored_row = converted_row.map(lambda x: score_mapping_pss.get(x, 0))

    """
    # Apply the reverse mapping to the specified columns
    for column in columns_to_reverse:
        if column in row:
            converted_row[column] = reverse_mapping.get(converted_row[column], converted_row[column])
    """
    # Calculate the total score
    total_score = converted_row.sum()

    total_score_forpt = scored_row.sum()

    print(f'Total score of Pss is {total_score}')

    max_possible_score = 40  # 10 questions * highest score per question (4)

    global pss_pt; pss_pt = (total_score_forpt / max_possible_score) * 100

    # Determine the stress category based on the total score
    if total_score <= 13:
        category = "Low stress"
    elif 14 <= total_score <= 26:
        category = "Moderate stress"
    elif 27 <= total_score <= 40:
        category = "High stress"
    else:
        category = "Unknown"

    return category, pss_pt

def calculate_activity(row,mapping,calorie_burnt):
    # Identify columns where 'active' was replaced with a number
    changed_columns = df.columns[df.apply(lambda col: col.astype(str).str.contains('Active').any())]

    for col_label in df.columns[20:36]:
        if (df[col_label] == 'Active').any():
            col_number_mod7 = df.columns.get_loc(col_label) % 8
            df[col_label] = df.apply(lambda row: col_number_mod7 if row[col_label] == 'Active' else row[col_label], axis=1)

    # Replace 'active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(36, 42)):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = df.columns[col_index]
        df[col_label] = df.apply(lambda row, col=col_label, inc=increment: inc if 'Active' in str(row[col]) and pd.notna(row[col]) else row[col], axis=1)


    #Calculate the product for the changed values (replaced 'active' with a number)
    df['Active_product'] = df.apply(lambda row: int(np.prod([float(row[col]) for col in changed_columns if isinstance(row[col], (int, float)) and not pd.isna(row[col])])), axis=1)


    # Identify columns where 'Somewhat active' was replaced with a number
    changed_columns = df.columns[df.apply(lambda col: col.astype(str).str.contains('Somewhat active').any())]

    for col_label in df.columns[:16]:
        if (df[col_label] == 'Somewhat active').any():
            col_number_mod7 = df.columns.get_loc(col_label) % 8
            df[col_label] = df.apply(lambda row: col_number_mod7 if row[col_label] == 'Somewhat active' else row[col_label], axis=1)

    # Replace 'Somewhat active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(16, len(df.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = df.columns[col_index]
        df[col_label] = df.apply(lambda row, col=col_label, inc=increment: inc if 'Somewhat active' in str(row[col]) and pd.notna(row[col]) else row[col], axis=1)


    #Calculate the product for the changed values (replaced 'Somewhat active' with a number)
    df['Somewhat active_product'] = df.apply(lambda row: int(np.prod([float(row[col]) for col in changed_columns if isinstance(row[col], (int, float)) and not pd.isna(row[col])])), axis=1)


    # Identify columns where 'Somewhat active' was replaced with a number
    changed_columns = df.columns[df.apply(lambda col: col.astype(str).str.contains('Moderately active').any())]

    for col_label in df.columns[:16]:
        if (df[col_label] == 'Moderately active').any():
            col_number_mod7 = df.columns.get_loc(col_label) % 8
            df[col_label] = df.apply(lambda row: col_number_mod7 if row[col_label] == 'Moderately active' else row[col_label], axis=1)

    # Replace 'Somewhat active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(16, len(df.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = df.columns[col_index]
        df[col_label] = df.apply(lambda row, col=col_label, inc=increment: inc if 'Moderately active' in str(row[col]) and pd.notna(row[col]) else row[col], axis=1)


    #Calculate the product for the changed values (replaced 'Somewhat active' with a number)
    df['Moderately active_product'] = df.apply(lambda row: int(np.prod([float(row[col]) for col in changed_columns if isinstance(row[col], (int, float)) and not pd.isna(row[col])])), axis=1)


    # Identify columns where 'Somewhat active' was replaced with a number
    changed_columns = df.columns[df.apply(lambda col: col.astype(str).str.contains('Sedentary').any())]

    for col_label in df.columns[:16]:
        if (df[col_label] == 'Sedentary').any():
            col_number_mod7 = df.columns.get_loc(col_label) % 8
            df[col_label] = df.apply(lambda row: col_number_mod7 if row[col_label] == 'Sedentary' else row[col_label], axis=1)

    # Replace 'Somewhat active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(16, len(df.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = df.columns[col_index]
        df[col_label] = df.apply(lambda row, col=col_label, inc=increment: inc if 'Sedentary' in str(row[col]) and pd.notna(row[col]) else row[col], axis=1)


    #Calculate the product for the changed values (replaced 'Somewhat active' with a number)
    df['Sedentary_product'] = df.apply(lambda row: int(np.prod([float(row[col]) for col in changed_columns if isinstance(row[col], (int, float)) and not pd.isna(row[col])])), axis=1)
    #multipy by MET score
    df['Active_product'] = df['Active_product'] * 8.0
    df['Somewhat active_product'] = df['Somewhat active_product'] * 3.3
    df['Moderately active_product'] = df['Moderately active_product'] * 4
    df['Sedentary_product'] = df['Sedentary_product'] * 1.3

    #Adding MET Score

    df['MET_score'] = df['Active_product'] + df['Somewhat active_product'] + df['Moderately active_product'] + df['Sedentary_product']

def categorize_activity_level(MET_score):
    if MET_score <= 600:
        return "Low"
    elif 600 <= MET_score <= 3000:
        return "Moderate"
    elif MET_score >= 3000:
        return "High"
    else:
        return "Unknown"


# For calorie burnt logic
def calculate_calorie_burnt(met, Weight):
    calorie_burnt = (met * 3.5 * Weight) / 200
    return calorie_burnt


#Paid Plan functions

def calculate_mpa_score(row, mpa_score_mapping):
    print(row)
    mpa_score = sum(mpa_score_mapping.get(str(response).strip().lower(), 0) for response in row)
    print(f'MPA is {mpa_score}')
    return mpa_score

def map_response_to_dosha(response, prakriti_mapping):

    if pd.isna(response):
        return 'Unknown'
    closest_match = process.extractOne(str(response).lower(), prakriti_mapping.keys(), score_cutoff=90)
    return prakriti_mapping.get(closest_match[0], 'No Match') if closest_match else 'No Match'

def determine_final_dosha(dosha_counts):

    most_common = dosha_counts.most_common(2)
    if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
        print(f"Dual Dosha found {most_common[0][0]} and {most_common[1][0]}, each found {most_common[0][1]} times")
        return f'Dual Dosha found with {most_common[0][0]} and {most_common[1][0]}'
    else:
        print(f"Most common Dosha: {most_common[0][0]}, found {most_common[0][1]} times")
        return most_common[0][0]


def total_score(row, mpa_score_mapping):
    print(f'Checking row {row}')

    # Function to standardize the response string
    def standardize_string(s):
        return str(s).strip().lower().replace("’", "'")

    # Standardize the mapping keys
    standardized_mapping = {standardize_string(key): value for key, value in mpa_score_mapping.items()}

    total = 0
    for response in row:
        standardized_response = standardize_string(response)
        response_score = standardized_mapping.get(standardized_response, 0)
        print(f'Score for response "{response}": {response_score}')
        total += response_score

    print(f'Score of this segment is {total}')
    return total


def major_vikriti(row, ayurveda_mapping):
    #print("Function called with row:", row)  # Print the row at the start
    dosha_counts = Counter()
    #print(f'Dosha count is {dosha_counts}')
    no_match_found = True

    # Function to standardize text for case-insensitivity
    def standardize_text(text):
        return text.strip().lower() if isinstance(text, str) else text

    #print("Ayurveda mapping keys:", ayurveda_mapping.keys())  # Print mapping keys

    for column, response in row.iteritems():
        standardized_column = standardize_text(column)
        #print(f'the question to be matched is {standardized_column}')
        standardized_response = standardize_text(response)

        #print(f"Processing: Column - {standardized_column}, Response - {standardized_response}")

        for question, answers in ayurveda_mapping.items():
            #print(f'Question is {question} and answer is {answers}')
            standardized_question = standardize_text(question)
            #print(f'the mapping question to be matched is {standardized_question}')
            standardized_answers = {standardize_text(key): value for key, value in answers.items()}

            if standardized_question in standardized_column:
                no_match_found = False
                dosha = standardized_answers.get(standardized_response)
                #print(f'dosha found is {dosha}')
                if dosha:
                    if isinstance(dosha, list):
                        dosha_counts.update(dosha)
                    else:
                        dosha_counts[dosha] += 1
                break

        if dosha_counts:
            dominant_dosha = max(dosha_counts, key=dosha_counts.get)
            print(f"Current dominant dosha: {dominant_dosha} with count: {dosha_counts[dominant_dosha]}")

    if no_match_found:
        print(f"No match found for question: {column}, answer: {response}")
        return ("No Match", 0)

    sorted_doshas = sorted(dosha_counts.items(), key=lambda x: x[1], reverse=True)
    doshas, counts = zip(*sorted_doshas) if sorted_doshas else ([], [])

    return doshas, counts


def calculate_activity(row):
    # Identify columns where 'active' was replaced with a number
    active_columns = row.columns[row.astype(str).apply(lambda col: col.str.contains("Active").any())]

    for col_label in row.columns[21:36]:
        if (row[col_label] == 'Active').any():
            col_number_mod7 = row.columns.get_loc(col_label) % 8
            row[col_label] = row.apply(lambda r: col_number_mod7 if r[col_label] == 'Active' else r[col_label], axis=1)

    # Replace 'active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(36, len(row.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = row.columns[col_index]
        row[col_label] = row.apply(lambda r, col=col_label, inc=increment: inc if 'Active' in str(r[col]) and pd.notna(r[col]) else r[col], axis=1)

    # Calculate the product for the changed values
    row['Active_product'] = row.apply(lambda r: int(np.prod([float(r[col]) for col in active_columns if isinstance(r[col], (int, float)) and not pd.isna(r[col])])), axis=1)

    # Identify columns where 'Somewhat active' was replaced with a number
    somewhat_active_columns = row.columns[row.astype(str).apply(lambda col: col.str.contains('Somewhat active').any())]

    for col_label in row.columns[21:36]:
        if (row[col_label] == 'Somewhat active').any():
            col_number_mod7 = row.columns.get_loc(col_label) % 8
            row[col_label] = row.apply(lambda r: col_number_mod7 if r[col_label] == 'Somewhat active' else r[col_label], axis=1)

    # Replace 'Somewhat active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(36, len(row.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = row.columns[col_index]
        row[col_label] = row.apply(lambda r, col=col_label, inc=increment: inc if 'Somewhat active' in str(r[col]) and pd.notna(r[col]) else r[col], axis=1)

    # Calculate the product for the changed values (replaced 'Somewhat active' with a number)
    row['Somewhat_active_product'] = row.apply(lambda r: int(np.prod([float(r[col]) for col in somewhat_active_columns if isinstance(r[col], (int, float)) and not pd.isna(r[col])])), axis=1)

    # Identify columns where 'Moderately active' was replaced with a number
    moderately_active_columns = row.columns[row.astype(str).apply(lambda col: col.str.contains('Moderately active').any())]

    for col_label in row.columns[21:36]:
        if (row[col_label] == 'Moderately active').any():
            col_number_mod7 = row.columns.get_loc(col_label) % 8
            row[col_label] = row.apply(lambda r: col_number_mod7 if r[col_label] == 'Moderately active' else r[col_label], axis=1)

    # Replace 'Moderately active' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(36, len(row.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = row.columns[col_index]
        row[col_label] = row.apply(lambda r, col=col_label, inc=increment: inc if 'Moderately active' in str(r[col]) and pd.notna(r[col]) else r[col], axis=1)

    # Calculate the product for the changed values (replaced 'Moderately active' with a number)
    row['Moderately_active_product'] = row.apply(lambda r: int(np.prod([float(r[col]) for col in moderately_active_columns if isinstance(r[col], (int, float)) and not pd.isna(r[col])])), axis=1)

    # Identify columns where 'Sedentary' was replaced with a number
    sedentary_columns = row.columns[row.astype(str).apply(lambda col: col.str.contains('Sedentary').any())]

    for col_label in row.columns[21:36]:
        if (row[col_label] == 'Sedentary').any():
            col_number_mod7 = row.columns.get_loc(col_label) % 8
            row[col_label] = row.apply(lambda r: col_number_mod7 if r[col_label] == 'Sedentary' else r[col_label], axis=1)

    # Replace 'Sedentary' with a sequence of 10 for columns 16 onwards
    for i, col_index in enumerate(range(36, len(row.columns))):
        increment = i * 10  # Start from 0 and increment by 10 for each column
        col_label = row.columns[col_index]
        row[col_label] = row.apply(lambda r, col=col_label, inc=increment: inc if 'Sedentary' in str(r[col]) and pd.notna(r[col]) else r[col], axis=1)

    # Calculate the product for the changed values (replaced 'Sedentary' with a number)
    row['Sedentary_product'] = row.apply(lambda r: int(np.prod([float(r[col]) for col in sedentary_columns if isinstance(r[col], (int, float)) and not pd.isna(r[col])])), axis=1)

    return row


def calculate_activity_products(activity_df):
    # Calculate the product for the changed values
    activity_df = calculate_activity(activity_df)

    # Multiply by MET score
    activity_df['Active_product'] = activity_df['Active_product'] * 8.0
    activity_df['Somewhat_active_product'] = activity_df['Somewhat_active_product'] * 3.3
    activity_df['Moderately_active_product'] = activity_df['Moderately_active_product'] * 4
    activity_df['Sedentary_product'] = activity_df['Sedentary_product'] * 1.3

    # Adding MET Score
    activity_df['MET_score'] = (
        activity_df['Active_product'] +
        activity_df['Somewhat_active_product'] +
        activity_df['Moderately_active_product'] +
        activity_df['Sedentary_product']
    )

    return activity_df

def categorize_activity_level(MET_score):
    if MET_score <= 600:
        return "Low"
    elif 600 <= MET_score <= 3000:
        return "Moderate"
    elif MET_score >= 3000:
        return "High"
    else:
        return "Unknown"

# For calorie burnt logic
def calculate_calorie_burnt(met, weight_kg):
    calorie_burnt = (met * 3.5 * weight_kg) / 200
    return calorie_burnt



def merge_results(prakriti_file, sleep_file, bmi_file, mpa_file, mwq_file, occ_stress, Pss_stress, vikriti_output_file, activity_file, lq_output_file, output_file):
    # Read each file into a DataFrame
    prakriti_df = pd.read_excel(prakriti_file)
    sleep_df = pd.read_excel(sleep_file)
    bmi_df = pd.read_excel(bmi_file)
    mpa_df = pd.read_excel(mpa_file)
    mwq_df = pd.read_excel(mwq_file)
    occ_stress_df = pd.read_excel(occ_stress)
    pss_stress_df = pd.read_excel(Pss_stress)
    activity_df = pd.read_excel(activity_file)
    vikriti_df = pd.read_excel(vikriti_output_file)
    lq_result_df = pd.read_excel(lq_output_file)


    # Merge the DataFrames on 'Email ID'
    merged_df = prakriti_df.merge(sleep_df, on='Email ID', how='left')
    merged_df = merged_df.merge(bmi_df, on='Email ID', how='left')
    merged_df = merged_df.merge(mpa_df, on='Email ID', how='left')
    merged_df = merged_df.merge(mwq_df, on='Email ID', how='left')
    merged_df = merged_df.merge(occ_stress_df, on='Email ID', how='left')
    merged_df = merged_df.merge(pss_stress_df, on='Email ID', how='left')
    merged_df = merged_df.merge(activity_df, on='Email ID', how='left')
    merged_df = merged_df.merge(vikriti_df, on='Email ID', how='left')
    merged_df = merged_df.merge(lq_result_df, on='Email ID', how='left')


    # Write the merged DataFrame to a new Excel file
    merged_df.to_excel(Complete_file, index=False)

    print('All results merged into', Complete_file)





def main():
    responses_file_path = 'KYWR_A.xlsx'
    Prakriti_file_path = 'Prakriti_discovered.xlsx'
    Sleep_output_file = 'Sleep_score.xlsx'
    BMI_file_path = 'BMI_Results.xlsx'
    MPA_output_file = 'MPA_output.xlsx'
    MWQ_output_file = 'MWQ_with_sections.xlsx'
    Occ_output_file = 'Occ_result.xlsx'
    Pss_output_file = 'Pss_result.xlsx'
    activity_file = 'activity_result.xlsx'
    vikriti_output_file = 'Vikriti_result.xlsx'

    lq_output_file = 'LQ_result.xlsx'


    responses_df = pd.read_excel('KYWR_A.xlsx')

    prakriti_responses_df = pd.read_excel(responses_file_path, usecols='BP:CD')

    sleep_df = pd.read_excel(responses_file_path, usecols='AR:AV')

    BMI_df = pd.read_excel(responses_file_path, usecols=['Email Address', 'Height - Select feet', 'Height - Select Inches', 'Weight in Kilograms'])

    emails_df = pd.read_excel(responses_file_path, usecols='B')  # Assuming column 'B' contains email IDs

    #Sleep code starts

    print('Starting Sleep calculation now.')

    sleep_df['Sleep Score'] = sleep_df.apply(calculate_sleep_score, axis=1)

    sleep_output_df = pd.concat([emails_df, sleep_df['Sleep Score']], axis=1)

    sleep_output_df.columns = ['Email ID', 'Sleep Score']

    sleep_output_df.to_excel(Sleep_output_file, index=False)

    print('Sleep score calculation completed and saved to', Sleep_output_file)

    print('.....')

    #BMI code starts here

    print('Now starting BMI calculation.')

    # Extracting height in feet and inches

    BMI_df['BMI'] = BMI_df.apply(lambda row: calculate_bmi(row['Height - Select feet'], row['Height - Select Inches'], row['Weight in Kilograms']), axis=1)

    BMI_output_df = BMI_df[['Email Address', 'BMI']]

    BMI_output_df.columns = ['Email ID', 'BMI']

    BMI_output_df.to_excel(BMI_file_path, index=False)

    print('BMI calculation completed and saved to', BMI_file_path)

    print('Starting with Prakriti Calculation.')

    #Prakriti calculation code

    mapped_responses = prakriti_responses_df.applymap(lambda x: map_response_to_dosha(x, prakriti_mapping))

    final_dosha = mapped_responses.apply(lambda row: determine_final_dosha(Counter(row)), axis=1)

    final_prakriti_df = pd.concat([emails_df, final_dosha], axis=1)

    final_prakriti_df.columns = ['Email ID', 'Final Dosha']

    final_prakriti_df.to_excel(Prakriti_file_path, index=False)

    print('Prakriti calculation done successfully.')

    print('Starting MPA calculation.')

    mpa_df = pd.read_excel(responses_file_path, usecols='I:L')

    mpa_df['MPA Score'] = mpa_df.apply(lambda row: calculate_mpa_score(row, mpa_score_mapping), axis=1)

    mpa_output_df = pd.concat([emails_df, mpa_df['MPA Score']], axis=1)

    mpa_output_df.columns = ['Email ID', 'MPA Score']

    mpa_output_df.to_excel(MPA_output_file, index=False)

    print('MPA score calculation completed and saved to', MPA_output_file)

    #starting MWQ Code
    print('Starting MWQ Code.')

    # Initialize the output DataFrame with email IDs
    output_df = emails_df.copy()
    output_df.columns = ['Email ID']

    for score_name, col_range in CATEGORY_RANGES.items():
        print(f'Starting {score_name} Calculation.')
        score_df = pd.read_excel(responses_file_path, usecols=col_range)
        score_df[score_name] = score_df.apply(lambda row: total_score(row, mwq_mappings), axis=1)
        output_df[score_name] = score_df[score_name]

    score_columns = list(CATEGORY_RANGES.keys())

    output_df['Total MWQ Score'] = output_df[score_columns].sum(axis=1)

    output_df['MWQ Percentage'] = (output_df['Total MWQ Score'] / 188) * 100

    output_df.to_excel(MWQ_output_file, index=False)

    print('All calculations completed and saved to', MWQ_output_file)

    print('Starting the Occ stress code.')


    # Read the Excel file
    occ_stress_df = pd.read_excel(responses_file_path, usecols='AX:BE')

    # Copy the DataFrame that contains email IDs (assuming it's named emails_df)
    occ_stress_results_df = emails_df.copy()

    # Apply the function and split the results into two separate columns
    occ_stress_results_df[['Occupational Stress Category', 'Occupational Percentage']] = occ_stress_df.apply( lambda row: calculate_occupational_stress(row, text_to_code_mapping, score_mapping_occupation), axis=1).apply(pd.Series)

    # Ensure the column names are correctly set
    occ_stress_results_df.columns = ['Email ID', 'Occupational Stress Category', 'Occupational Percentage']

    # Save the results
    occ_stress_results_df.to_excel(Occ_output_file, index=False)

    print('Occupational stress calculations completed and saved to', Occ_output_file)

    print('Starting the Pss stress code.')

    pss_stress_df = pd.read_excel(responses_file_path, usecols='BF:BO')

    pss_stress_result_df = emails_df.copy()

    pss_stress_result_df[['Percieved Stress Category', 'Pss Percentage']] = pss_stress_df.apply( lambda row: calculate_perceived_stress(row, text_to_code_mapping, score_mapping_pss), axis=1).apply(pd.Series)

    pss_stress_result_df.columns = ['Email ID', 'Perceived Stress Category', 'Pss Percentage']

    pss_stress_result_df.to_excel(Pss_output_file, index=False)

    print('Perceived stress calculations complete and saved to', Pss_output_file)


    # Vikriti Code Starts

    print('Starting Vikriti calculation.')


    Vikriti_df = pd.read_excel(responses_file_path, usecols='DZ:ES')

    Vikriti_df[['Dominant Vikriti', 'Dosha Counts']] = Vikriti_df.apply(lambda row: major_vikriti(row, ayurveda_mapping), axis=1, result_type='expand')

    Vikriti_output_df = pd.concat([emails_df, Vikriti_df[['Dominant Vikriti', 'Dosha Counts']]], axis=1)

    Vikriti_output_df.columns = ['Email ID', 'Dominant Vikriti', 'Dosha Counts']

    Vikriti_output_df.to_excel(vikriti_output_file, index=False)

    print('Vikriti score calculation completed and saved to', vikriti_output_file)

    print('Starting the activity code')
    # Read responses_file_path into a DataFrame
    activity_df = pd.read_excel(responses_file_path, usecols='V:AQ,H')
    #activity_df = emails_df.copy()

    # Check if 'Weight (kg)' column exists in responses_df
    if 'Weight in Kilograms' in responses_df.columns:
        # Calculate activity and activity products
        activity_df = calculate_activity(responses_df)

        activity_df = calculate_activity_products(activity_df)

        # Calculate MET_score based on the provided formula
        activity_df['MET_score'] = (
            activity_df['Active_product'] +
            activity_df['Somewhat_active_product'] +
            activity_df['Moderately_active_product'] +
            activity_df['Sedentary_product']
        )

        global met_score; met_score = activity_df['MET_score']

        print(f'the met score is {met_score}')

        global met_score_pt; met_score_pt = (met_score/10000) * 100


        # Add 'Weight (kg)' from responses_df to activity_df
        activity_df['Weight in Kilograms'] = responses_df['Weight in Kilograms']

        # Calculate Activity Level and Calorie Burnt
        activity_df['Activity_Level'] = activity_df['MET_score'].apply(categorize_activity_level)
        activity_df['Calorie_Burnt'] = activity_df.apply(lambda row: calculate_calorie_burnt(row['MET_score'], row['Weight in Kilograms']), axis=1)

        # Reorder columns

        activity_df = pd.concat([emails_df, activity_df[['Activity_Level', 'Calorie_Burnt', 'MET_score']]], axis=1)

        activity_df.columns = ['Email ID', 'Activity_Level', 'Calorie_Burnt', 'MET_score']

        # Save to Excel
        activity_df.to_excel(activity_file, index=False)

        print('Activity calculations complete and saved to', activity_file)
    else:
        print('Error: "Weight in Kilograms" column not found in responses_df')

    print("Data types of 'Email ID' in each DataFrame:")
    print("Prakriti DataFrame:", final_prakriti_df['Email ID'].dtypes)
    print("Sleep DataFrame:", sleep_output_df['Email ID'].dtypes)
    print("BMI DataFrame:", BMI_output_df['Email ID'].dtypes)

    #Starting LQ Code

    print('Starting code for current version of LQ, which basically makes little sense.')

    print(f'Sleep % is {sleep_pt}')

    print(f'occ is {occ_pt}')

    print(f'pss is {pss_pt}')

    print(f'met is {met_score_pt}')

    total_pt = sleep_pt + occ_pt + pss_pt + met_score_pt

    LQ = total_pt/4

    print(f'LQ is {LQ}')

    # Create LQ DataFrame
    lq_df = pd.DataFrame({'LQ': [LQ]})

    lq_result_df = pd.concat([emails_df, lq_df], axis=1)

    lq_result_df.columns = ['Email ID', 'LQ Percentage']

    lq_result_df.to_excel(lq_output_file, index=False)

    print('LQ calculations complete and saved to', lq_output_file)


    # Merge the results
    merge_results(Prakriti_file_path, Sleep_output_file, BMI_file_path, MPA_output_file, MWQ_output_file, Occ_output_file, Pss_output_file, vikriti_output_file, activity_file, lq_output_file, Complete_file)

if __name__ == "__main__":
    main()


Starting Sleep calculation now.
Sleep score calculation completed and saved to Sleep_score.xlsx
.....
Now starting BMI calculation.
the height is 1.7272 in meters
The weight is 53
BMI is 17.766038992285594
BMI calculation completed and saved to BMI_Results.xlsx
Starting with Prakriti Calculation.
Most common Dosha: Pitta, found 10 times
Prakriti calculation done successfully.
Starting MPA calculation.
Do you experience pain during periods?                                                                                  Usually (on most occasions)
Where do you feel discomfort during your period? Please specify the areas of your body where you experience pain    Groin area, Lower back, Abdomen
On a scale from mild to severe, how would you describe the intensity of your pain during periods?                                          Moderate
What is the average duration of the pain?                                                                                                  1-2 days
Nam

<ipython-input-4-53c078e358c5>:574: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, response in row.iteritems():


the met score is 0    16.6
Name: MET_score, dtype: float64
Activity calculations complete and saved to activity_result.xlsx
Data types of 'Email ID' in each DataFrame:
Prakriti DataFrame: object
Sleep DataFrame: object
BMI DataFrame: object
Starting code for current version of LQ, which basically makes little sense.
Sleep % is 0.0
occ is 6.25
pss is 37.5
met is 0    0.166
Name: MET_score, dtype: float64
LQ is 0    10.979
Name: MET_score, dtype: float64
LQ calculations complete and saved to LQ_result.xlsx
All results merged into Complete_results_file_nov28.xlsx
